# Implementing a custom fitter

For when fit_lc() won't cut it.

In [ ]:
from __future__ import print_function

import os

import numpy as np
import sncosmo
import matplotlib as mpl

%matplotlib inline

mpl.rc('savefig', dpi=110.)

In [ ]:
# we're going to use an optimizer from scipy
from scipy.optimize import fmin_l_bfgs_b

In [ ]:
model = sncosmo.Model(source='salt2')
data = sncosmo.read_lc('data/lc-SDSS19230.list', format='salt2')
print(data)

In [ ]:
model.set(z=data.meta['Z_HELIO'])

In [ ]:
# Define an objective function that we will pass to the minimizer.
# The function arguments must comply with the expectations of the specfic
# minimizer you are using.

def objective(parameters, model, data):
    model.parameters[1:5] = parameters
    
    # evaluate model fluxes at times/bandpasses of data
    model_flux = model.bandflux(data['Filter'], data['Date'],
                                zp=data['ZP'], zpsys=data['MagSys'])

    # calculate and return chi^2
    return np.sum(((data['Flux'] - model_flux) / data['Fluxerr'])**2)

In [ ]:
# starting parameter values in same order as `model.param_names`:
start_parameters = [54400., 1e-5, 0., 0.]  # t0, x0, x1, c

# parameter bounds in same order as `model.param_names`:
bounds = [(54356., 54433.), (None, None), (-3., 3), (-0.3, 0.3)]

parameters, val, info = fmin_l_bfgs_b(objective, start_parameters, args=(model, data),
                                      bounds=bounds, approx_grad=True)

In [ ]:
print(parameters)

In [ ]:
print(val)

In [ ]:
print(info)

In [ ]:
# plot it
sncosmo.plot_lc(data, model);